### Functional Classification and Sequence Homology Across Primate Genomes
**Emily M. Liau**

#### Introduction

The divergence of human and chimpanzee ancestors dates back to at least 6.5-7.5 million years ago. As a result of their homology, humans and chimpanzees share a large number of conserved sequences. Although recent technologies have improved, it remains difficult to quantify the exact differences between human and chimpanzee genomes based on protein-coding sequences alone. Therefore, identifying the major structural and regulatory genetic alterations that have shaped the biological function of specific DNA sequences over time could provide insight into human and chimpanzee speciation.

In this analysis, we seek to classify DNA sequences across primate genomes based on their apparent biological function. More specifically, we will analyze DNA sequences from humans and chimpanzees, and attempt to classify these sequences into seven main gene families: 
- **G-protein Coupled Receptors:** membrane proteins used by eukaryotic cells to convert extracellular signals into intracellular responses, such as responses to hormones, neurotransmitters, and external signals
- **Tyrosine Kinase:** enzymes that catalyze the phosphorylation of specific tyrosine residues in target proteins
- **Tyrosine Phosphatase:** enzymes that utilize ATP to regulate signal transduction pathways by catalyzing the dephosphorylation of tyrosine-phosphorylated proteins to reverse the phosphorylation of tyrosine residues
- **Synthetase:** enzymes that catalyze the linking of two molecules using the energy derived from the separation of a pyrophosphate group from a triphosphate
- **Synthase:** enzymes that catalyze the synthesis of new compounds in the body, regardless of energy source
- **Ion Channel:** integral membrane proteins in mammaliam cells that form pores to allow the transport of specific ions by passive diffusion
- **Transcription Factor:** proteins involved in the process of transcribing DNA into RNA

The original dataset contains sequences from both human and chimpanzee genomes, as well as class values that correspond to the functions of the sequences. We apply a Multinomial Naive Bayes Classifier to predict the gene family that characterizes each sequence across both primate species. Comparing the resulting classifications across both primate species of interest will provide an additional perspective into the impact of evolution on sequence homology between humans and chimpanzees over time.

In [ ]:
# import requisite packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# function to output bold text
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

#### Data Processing - Human DNA Sequences

In the code segment below, we import the human DNA sequences, along with class values that correspond to each sequence's gene family/biological function. In addition, we calculate the observed counts for each gene family in our dataset of human DNA sequences.

In [ ]:
humanDF = pd.read_table('human.txt')
printmd("**DNA Sequences and Gene Classes in Humans**\n")
print(humanDF)
print('\n')

humanFrequencies = humanDF.groupby('class').count()
gene_family = ['G-protein Coupled Receptors', 'Tyrosine Kinase', 'Tyrosine Phosphatase', 'Synthetase', 'Synthase', 
               'Ion Channel', 'Transcription Factor']
humanFrequencies.insert(0, "gene family", gene_family)
humanFrequencies = humanFrequencies.rename(columns = {'sequence':'count'})
printmd('**Observed Counts of Gene Families in Humans**\n')
with pd.option_context('display.max_colwidth', None):
    print(humanFrequencies)

**Explanation:** Overall, the results in the human dataframes are consistent with the prevalence of certain biological pathways present in humans. For example, we can conclude based on the dataframe above, that the proportion of human DNA sequences that code for transcription factors outweighs the proportions present in other categories of gene families. We should expect that transcription factors are more prevalent compared to other categories of gene families, since they are the primary components responsible for regulating gene expression and producing functional protein products in eukaryotes. In addition, gene families such as G-protein coupled receptors (GPCRs) and synthetases are required in a fairly broad range of human biological pathways compared with less prevalent gene families such as tyrosine phosphatases. For example, tyrosine phosphatases are primarily utilized in physiological pathways, whereas GPCRs are used in a wider variety of processes in the human body, including those related to vision, olfaction, and the central nervous system. Therefore, the counts presented in the dataframe parallel the true frequencies that would be observed in biological pathways in humans.

#### Data Processing - Chimpanzee DNA Sequences

In the code segment below, we import the chimpanzee DNA sequences, along with class values that correspond to each sequence's gene family/biological function. In addition, we calculate the observed counts for each gene family in our dataset of chimpanzee DNA sequences.

In [ ]:
chimpDF = pd.read_table('chimp.txt')
printmd("**DNA Sequences and Gene Classes in Chimpanzees**\n")
print(chimpDF)
print('\n')

chimpFrequencies = chimpDF.groupby('class').count()
gene_family = ['G-protein Coupled Receptors', 'Tyrosine Kinase', 'Tyrosine Phosphatase', 'Synthetase', 'Synthase', 
               'Ion Channel', 'Transcription Factor']
chimpFrequencies.insert(0, "gene family", gene_family)
chimpFrequencies = chimpFrequencies.rename(columns = {'sequence':'count'})
printmd('**Observed Counts of Gene Families in Chimpanzees**\n')
with pd.option_context('display.max_colwidth', None):
    print(chimpFrequencies)

**Explanation:** Similar to the human data presented above, the results in the chimpanzee dataframes are consistent with the prevalence of certain biological pathways present in chimpanzees. Akin to the proportions of gene families observed in humans, ion channels appear to have the lowest frequency and transcription factors appear to have the highest frequency across all gene families in chimpanzees. We can conclude that the results presented in the chimpanzee dataframe parallel the relative proportions observed in humans, likely due to the prevalence of conserved sequences in both species across evolutionary time. In addition, transcription factors serve as a potential source of variability in gene expression between humans and chimpanzees. Since transcription factors are evolutionarily stable and structurally well-conserved, specific transription factors could be responsible for driving major changes between both primate species of interest, which serves as an explanation as to why transcription factors are the most prevalent gene family in both datasets. Overall, we can conclude that the counts presented in the dataframe parallel the true frequencies that would be observed in biological pathways in chimpanzees.

#### Comparing Proportions of Gene Families Across Humans and Chimpanzees

In the code segment below, we convert the gene family counts into proportions, since the number of observations in the human dataframe and the chimpanzee dataframe are not equivalent. Then, we merge the human and chimpanzee dataframes, in order to prepare the data for visualization.

In [ ]:
combined = pd.merge(humanFrequencies, chimpFrequencies, on = 'gene family')
combined = combined.rename(columns = {'count_x':'human', 'count_y':'chimp'})

# calculate observed proportions of gene families in humans and chimpanzees
for index, row in combined.iterrows():
    combined['human'] = combined['human'] / combined['human'].sum()
    combined['chimp'] = combined['chimp'] / combined['chimp'].sum()
printmd('**Proportions of Gene Families in Humans and Chimpanzees**\n')
print(combined)

In the code segment below, we develop a grouped bar chart in order to better visualize the relative proportions of each gene family classification between humans and chimpanzees.

In [ ]:
combined.plot.bar(x = 'gene family', logy = False)
plt.xticks(rotation = 80)
plt.legend(bbox_to_anchor = (1.05, 1), loc = 'upper left')
plt.xlabel('Gene Family')
plt.ylabel('Proportion')
plt.title('Sequence Classification Across Gene Families')
plt.show()

As presented in the merged dataframe and the grouped bar chart visualization above, the relative proportions of sequences that correspond to each gene family are fairly similar across the human and chimpanzee genomes, due to sequence conservation over evolutionary history. These results are expected, since humans and chimpanzees share 98.8% of their DNA, as a product of their descent from a relatively recent common ancestor.

#### Curating the k-mer Library

Since DNA sequences provide instructions to encode proteins and other essential molecules within orgamisms, we use k-mer counting to account for all potential open reading frames (ORFs) within each sequence. In this analysis, we use k-mers of size 6, since this size increases the chance that all the k-mers will overlap within sequence assembly, while also minimizing information loss. The function below splits the DNA sequences into hexamers, or k-mers of a pre-defined length of six nucleotides.

k-mer: all of a sequence's subsequences of length k

In [ ]:
def kmers(sequence, size = 6):
    return [sequence[x: x + size].lower() for x in range(len(sequence) - size + 1)]

In the code segment below, we use the 'kmers' function defined above in order to split each human DNA sequence into hexamers.

In [ ]:
humanDF['hexamers'] = humanDF.apply(lambda x: kmers(x['sequence']), axis = 1)
humanDF = humanDF.drop('sequence', axis = 1)
printmd("**Hexamers and Gene Classes in Humans**")
print(humanDF)

In the code segment below, we use the 'kmers' function defined above in order to split each chimpanzee DNA sequence into hexamers.

In [ ]:
chimpDF['hexamers'] = chimpDF.apply(lambda x: kmers(x['sequence']), axis = 1)
chimpDF = chimpDF.drop('sequence', axis = 1)
printmd("**Hexamers and Gene Classes in Chimpanzees**")
print(chimpDF)

In the code segment below, we convert the human hexamers into a list of string sentences, that will later be used as input for CountVectorizer. In addition, we create a separate variable to store the human class labels.

In [ ]:
human_texts = list(humanDF['hexamers'])
for item in range(len(human_texts)):
    human_texts[item] = ' '.join(human_texts[item])
human_class_labels = humanDF.iloc[:, 0].values
printmd('**Human Class Labels**\n')
print(human_class_labels)
printmd('**Hexamers in Humans**\n')
print(human_texts[0])

In the code segment below, we convert the chimpanzee hexamers into a list of string sentences, that will later be used as input for CountVectorizer. In addition, we create a separate variable to store the chimpanzee class labels.

In [ ]:
chimp_texts = list(chimpDF['hexamers'])
for item in range(len(chimp_texts)):
    chimp_texts[item] = ' '.join(chimp_texts[item])
chimp_class_labels = chimpDF.iloc[:, 0].values
printmd('**Chimpanzee Class Labels**\n')
print(chimp_class_labels)
printmd('**Hexamers in Chimpanzees**\n')
print(chimp_texts[0])

In the code segment below, we create the bag-of-words model, in which a certain text is represented as the bag of its words. Such a model disregards grammer and word order, but maintains multiplicity (one sentence, word, or phrase has more than one possible meaning). After creating the bag-of-words model, we apply CountVectorizer, a tool used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.

We apply this model to both the human and chimpanzee sequences using an n-gram value of 4, since increasing the value of n will improve the generated output text, while maintaining substantial difference from the training set.

n-gram: all combinations of adjacent words or letters of length n that can be found within the source text

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range = (4, 4))
x_human = cv.fit_transform(human_texts)
x_chimp = cv.transform(chimp_texts)

#### Classification

In order to classify the sequences into their respective gene families, we apply a Multinomial Naive Bayes Classifier, a method suitable for classification on datasets with discrete features. This approach predicts the tag of a certain text using the Bayes Theorem; by calculating each class' likelihood for a given sample, the Multinomial Naive Bayes Classifier will determine the tag with the greatest predicted probability. In this analysis, the tag corresponds to the gene family, and the input text consists of the hexamer sequences.

#### Human Multinomial Naive Bayes Classifier

In the code segment below, we train the Multinomial Naive Bayes Classifier on the 80% of the original dataset of human sequences, and test the classifier on the remaining 20% of the human data. In addition, we build this classifier with an alpha value of 0.1 in order to address the issue of zero probability in the Naive Bayes algorithm.

In [ ]:
from sklearn.model_selection import train_test_split
x_human_train, x_human_test, y_human_train, y_human_test = train_test_split(x_human, human_class_labels, test_size = 0.20, random_state = 42)

from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha = 0.1)
classifier.fit(x_human_train, y_human_train)
y_human_pred = classifier.predict(x_human_test)

The following function retrieves the accuracy, precision, recall, and F1 values for both the human and chimpanzee classifiers.

In [ ]:
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average = 'weighted')
    recall = recall_score(y_test, y_predicted, average = 'weighted')
    f1 = f1_score(y_test, y_predicted, average = 'weighted')
    return accuracy, precision, recall, f1

In order to visualize and summarize the performance of our Multinomial Naive Bayes classifier on the human sequences, we construct a confusion matrix, and output the accuracy, precision, recall, and F1 values.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
printmd("**Confusion Matrix - Human Multinomial Naive Bayes Classifier**\n")
print(pd.crosstab(pd.Series(y_human_test, name = 'Observed'), pd.Series(y_human_pred, name = 'Expected')))

accuracy, precision, recall, f1 = get_metrics(y_human_test, y_human_pred)
print("\nAccuracy = %.3f \nPrecision = %.3f \nRecall = %.3f \nF1 = %.3f" % (accuracy, precision, recall, f1))

Based on the output above, we can conclude that the accuracy of our Multinomial Naive Bayes Classifier on the human sequences is approximately 98.4% on the testing set. Based on our confusion matrix, we can conclude that our classifier accurately predicts the gene families present in humans based on sequences alone. However, in cases where class = 6 (transcription factor), the classifier is not always able to accurately characterize sequences that function as transcription factors.

#### Chimpanzee Multinomial Naive Bayes Classifier

In the code segment below, we train the Multinomial Naive Bayes Classifier on the 80% of the original dataset of chimpanzee sequences, and test the classifier on the remaining 20% of the chimpanzee data. In addition, we build this classifier with an alpha value of 0.1 in order to address the issue of zero probability in the Naive Bayes algorithm.

In [ ]:
from sklearn.model_selection import train_test_split
x_chimp_train, x_chimp_test, y_chimp_train, y_chimp_test = train_test_split(x_chimp, chimp_class_labels, test_size = 0.20, random_state = 42)

from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha = 0.1)
classifier.fit(x_chimp_train, y_chimp_train)
y_chimp_pred = classifier.predict(x_chimp_test)

In order to visualize and summarize the performance of our Multinomial Naive Bayes classifier on the chimpanzee sequences, we construct a confusion matrix, and output the accuracy, precision, recall, and F1 values.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
printmd("**Confusion Matrix - Chimpanzee Multinomial Naive Bayes Classifier**\n")
print(pd.crosstab(pd.Series(y_chimp_test, name = 'Observed'), pd.Series(y_chimp_pred, name = 'Expected')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average = 'weighted')
    recall = recall_score(y_test, y_predicted, average = 'weighted')
    f1 = f1_score(y_test, y_predicted, average = 'weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_chimp_test, y_chimp_pred)
print("\nAccuracy = %.3f \nPrecision = %.3f \nRecall = %.3f \nF1 = %.3f" % (accuracy, precision, recall, f1))

Based on the output above, we can conclude that the accuracy of our Multinomial Naive Bayes Classifier on the chimpanzee sequences is approximately 91.4% on the testing set. Based on our confusion matrix, we can conclude that our classifier accurately predicts the gene families present in chimpanzees based on sequences alone. However, in cases where class = 6 (transcription factor), the classifier is not always able to accurately characterize sequences that function as transcription factors.

#### Conclusion

As a result of our analysis, we can conclude that our Multinomial Naive Bayes Classifiers for both the human and chimpanzee datasets are able to accurately predict biological function from DNA sequences alone. With regards to specific insight this analysis provides, we can apply these results towards other studies to assess sequence homology across the human and chimpanzee genomes from the time of their descent from a common ancestor, which would allow us to better understand how evolution has played a role in altering gene function and regulation in both species. 

In the future, more extensive analysis on additional gene families, or on other regions of the genome, such as non-coding regions or mitochondrial DNA, could provide greater insight into the evolutionary change that has shaped the development of humans and chimpanzees over time.